# FIT5196 Assignment 1

#### Student Name:  Shihan Zhang

#### Student ID:        31268102

Date: 13/11/2020

Version: 5.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

### Work Flow for This task

1. First filter the text to remove non-English

2. merg all text from the single sheet to a dictionary `new_dict`

3. Use regx to grab words

4. Use the value in `new_dict` to generate primary uni gram -> `raw_uni`

5. De-duplicate words in `raw_uni` "page break/single day", and then calculate document frequncy

6. Remove the token which has (60 > document frequncy > 5) in `raw_uni` then get `unigramList`
    
7. Process `raw_vocab` with stopword to remove stopword, then process tokens with`PorterStemmer()`. 

8. Select top 100 words (single day) -> `final_uni`then writte into `.txt`.

9. Use `raw_vocab` to generate -> `raw_bigram`.. Find the top 100 biggram (single day) and write it to `.txt`

10. Find the 200 meaningful bigrams with PIM measure, then add to the `fial_vocab`

11. Generate matrix through `final_cleaned_vocab{}` and write vector to `.txt`

# library

In [29]:
from itertools import chain
import langid
import nltk
import pandas as pd
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer

## Initialization the basic parameters
- list `stopwords` store for stopword
- `regexpStr` the provided regex pattern
- `default_tokenizer` the regexToknizeer

In [30]:
# init list
stopwords_list = []
# init tokenizer
regexpStr = r"[a-zA-Z]+(?:[-\'][a-zA-Z]+)?"
default_tokenizer = RegexpTokenizer(regexpStr)

## Load source file
 - use pandas library to load excel file
 - `stopwords_list` to store loaed stopwords

In [32]:
excel = pd.ExcelFile('./31268102.xlsx')
with open("./stopwords_en.txt", 'r') as f:
    stopwords_list = f.read().splitlines()

# remove the duplicated value for stopword 
stopwords_set = set(stopwords_list)

## Initialization the Component functions
 - pass data tp `sortFreDict()`to get the sorted dictionary value
 - pass data and savePath to `saved()` to wrtie `.txt` file

In [33]:
# function to sort all word for final output
def sortFreqDist(data):
    res = []
    for key, value in data.items():
        # print(key, value)
        temp = (key, value)
        res.append(temp)
    # Sort elments by frequency
    result = sorted(res, key=lambda x: int(x[1]), reverse=True)
    return result

In [34]:
def saved(data: str, filePath, mode='w'):
    with open(filePath, f'{mode}') as f:
        f.write(data)

## Read single day/sheet 
from workflow:
1. First filter the text to remove non-English
2. merg all text from the single sheet to a dictionary `new_dict`

In [ ]:
# init dictionary for store single sheet data
new_dict = {}

# loop the all the sheets in the excel file
for i in range(len(excel.sheet_names)):
    # instant list for store text
    content = []
    # parse sheet by index
    df = excel.parse(i)
    # Delete columns that contain all NaN values
    df.dropna(1, how='all', inplace=True)
    # Delete all rows with NaN values
    df.dropna(axis=0, how='all', inplace=True)
    # rename columns by first row in the df
    df.rename(columns=df.iloc[0], inplace=True)
    # drop the frist row
    df = df[1:]
    # reset row index
    df.index = range(len(df.index))
    
    # filter out None English text
    for k in range(len(df.values)):
        # conver elemnt to string
        text = str(df.values[k][0])
        # detect if is English
        if (langid.classify(text)[0]) == 'en':
            # append English text to list
            content.append(text)
    
    # join all the text value(from every single sheet) to dictionary
    # the key for this dictionary is "sheet name"
    new_dict[excel.sheet_names[i]] = '\n'.join(map(str, content))

# print(new_dict.keys())

## Find unigram
from workflow:
3. Use regexTokenizer to grab words
4. Use the value in `new_dict` to generate primary uni gram -> `raw_uni`

In [22]:
def find_unigram(data):
    # init the dictionary to store raw unigram
    raw_uni = {}
    # loop the input dictionary key
    for i in data.keys():
        # extract the text content
        words = data[i]
        # Use tokenizer to capture words and convert them to lowercase
        unigram_token = default_tokenizer.tokenize(words.lower())
        # Use this dictionary to integrate all unigrams of dates
        # with date as Key and unigram token as value
        raw_uni[i] = unigram_token

    return raw_uni

## Procesee the clean unigram and vocabulary
from workflow:
5. De-duplicate words in `raw_uni` "page break/single day", and then calculate document frequncy

6. Remove the token which has (60 > document frequncy > 5) in `raw_uni` then get `unigramList`

In [ ]:
# import Stemmer
from nltk.stem import PorterStemmer
p = PorterStemmer()
# use un-proccess text to generate raw_vocab 
raw_vocab = find_unigram(new_dict)

In [19]:
# function for generate clean vocabulary and clean unigram
def unigramToVocab():
    # Combine all words and remove duplicate words
    # Only the daily words are deduplicated, and the merged words are not deduplicated
    doc_words = list(chain.from_iterable([set(value) for value in raw_vocab.values()]))
    # print('[doc_words]', len(doc_words))
    
    # Calculation frequency
    single_doc_freq = FreqDist(doc_words)

    # list to store the words that meet the conditions(document frequncy)
    includedWords = []
    
    # find the document frequncy
    # only keep the words that meet the threshold requirement
    for word, freq in single_doc_freq.items():
        if 5 <= freq <= 60:
            includedWords.append(word)
    
    # dictionary to store the processed words
    cleaned_vocab = {}
    
    for date, values in raw_vocab.items():
        temp = []
        for word in values:
            # Remove stop words; filter data that dont meet threshold requirement
            if word in includedWords and word not in stopwords_set:
                new_word = p.stem(word)
                # keep the words which length less than 3
                if len(new_word) > 2:
                    temp.append(word)
        
        # store the clean vocab to dictionary
        cleaned_vocab[date] = temp
    
    # generated processed unigrame
    unigramList = set(list(chain.from_iterable(cleaned_vocab.values())))
    
    return unigramList, cleaned_vocab

## getting unigram text
from workflow:
7. Process `raw_vocab` with stopword to remove stopword, then process tokens with`PorterStemmer()`. 

8. Select top 100 words (single day) -> `final_uni`then writte into `.txt`.

In [ ]:
def genUniTxt():
    path = './31268102_100uni.txt'
    final_uni = ''
    # get un-process unigram token
    for date, words in raw_vocab.items():
        each_words = []
        for item in words:
            # the raw word should not remove stopwords
            if item in stopwords_set:
                continue
            # use prot stem words
            new_word = p.stem(item)
            if new_word not in stopwords_set:
                each_words.append(new_word)
        sorted_words = sortFreqDist(FreqDist(each_words))
        # reformat all processed unigram to string, use date as the header
        if not final_uni:
            # first line
            final_uni = str(date) + ':' + str(sorted_words[:100])
        else:
            final_uni = final_uni + '\n' + str(date) + ':' + str(sorted_words[:100])
    
    # write to .txt file       
    saved(final_uni, path)

## getting bigram text
from workflow:
9. Use `raw_vocab` to generate -> `raw_bigram` Find the top 100 biggram (single day) and write it to `.txt`

In [ ]:
def genBiText():
    path = './31268102_100bi.txt'
    final_uni = ''
    for date, words in raw_vocab.items():
        # find bigram from raw_unit
        raw_bigram = nltk.collocations.BigramCollocationFinder.from_words(words)
        # sorted by frequncy
        final_bigram = sortFreqDist(raw_bigram.ngram_fd)
        
        # reformat all processed bigram to string, use date as the header
        if not final_uni:
            # frist line
            final_uni = str(date) + ':' + str(final_bigram[:100])
        else:
            final_uni = final_uni + '\n' + str(date) + ':' + str(final_bigram[:100])
    
    # write to .txt file
    saved(final_uni, path)

## get 200 meaningful bigrams
from workflow:
10. Find the 200 meaningful bigrams with PIM measure, then add to the `fial_vocab`

In [ ]:
def genVocab():
    all_tweet_words = list(chain.from_iterable(raw_vocab.values()))
    # find bigram from the raw unigram
    raw_bigram = nltk.collocations.BigramCollocationFinder.from_words(all_tweet_words)
    # get bigram with measures PMI 200
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    # get 200 meaningful words
    top_200_bigrams = raw_bigram.nbest(bigram_measures.pmi, 200)
    
    # reformat the bigram with '_'
    target_bigrams = []
    for item in top_200_bigrams:
        temp = item[0] + '_' + item[1]
        target_bigrams.append(temp)
    # return formated bigram
    return target_bigrams

### get Final vocabulary
1. get vocabulary(unigram only) from processed unigram(removd context-denpend/indendent words)
2. get 200 meaningful bigrams
3. combine the 1 & 2 to get full vocabulary list

In [ ]:
# get process unigram and vocab form 'unigramToVocab()'
unigramList, final_cleaned_vocab = unigramToVocab()

# get 200 meaningful word from 'genVocab()'
target_bigrams = genVocab()

# get final vocabulary 
final_vocab = list(unigramList)
# combine the 200 bigrams
final_vocab.extend(target_bigrams)

# dictionary to store vocabulary index
word_vocab_index = {}

## wrtie final Vocabulary to .txt file

In [ ]:
def sample_vocab():
    # reomve any protential duplicate value
    new_word_list = set(final_vocab)
    result = list(new_word_list)
    # sort vocabulary words
    result.sort(reverse=False)

    path = './31268102_vocab.txt'
    vocab_text = ''
    # reformat the words with index number
    for index, item in enumerate(result):
        # formatting with ": index" 
        temp = item + ':' + str(index)
        if not vocab_text:
            vocab_text = temp
        else:
            vocab_text = vocab_text + '\n' + temp
        # detect bigrames
        # reassgin the bigrame index(prevent all unigrams in front fo the file)
        if '_' in item:
            word_list = item.split('_')
            word_vocab_index[word_list[0]] = index
            word_vocab_index[word_list[1]] = index
        else:
            word_vocab_index[item] = index
            
    # write to .txt file
    saved(vocab_text, path)

# run function 
sample_vocab()

## generatting vector
11. Generate matrix through `final_cleaned_vocab{}` and write vector to `.txt`

In [ ]:
def getVect():
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(analyzer="word", token_pattern=regexpStr)
    # get data from final clean vocabulary
    data_features = vectorizer.fit_transform([' '.join(value) for value in final_cleaned_vocab.values()])
    vocabTwo = vectorizer.get_feature_names()
    firstArray = data_features.toarray()
    # get date form vocabulary
    dateList = [date for date in final_cleaned_vocab.keys()]
    
    countVectText = ''
    for index, date in enumerate(dateList):
        temp = str(date)
        for word, count in zip(vocabTwo, firstArray[index]):
            if count > 0:
                wordIndex = word_vocab_index.get(word)
                temp = temp + ',' + str(wordIndex) + ":" + str(count)
        if not countVectText:
            countVectText = temp + '\n'
        else:
            countVectText = countVectText + temp + '\n'

    path = './31268102_countVec.txt'
    saved(countVectText, path)


getVect()